BloomTech Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets

- [ ] Continue to clean and explore your data. 
- [ ] For the evaluation metric you chose, what score would you get just by guessing?
- [ ] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

In [6]:
%%capture
import sys

# For Google reproducibility
if 'google.colab' in sys.modules:
    DATA_PATH = "/content/drive/My Drive/Kaggle"
    !pip install category_encoders==2.*

    #Connect to remote data
    from google.colab import drive
    drive.mount(DATA_PATH, force_remount=True)

# Local data store on drive D:
else:
    DATA_PATH = "D:/Datafiles/"

In [26]:
# Import Block
import pandas as pd
import numpy as np
# #imblearn
# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.over_sampling import RandomOverSampler
#sklearn
from sklearn.metrics import accuracy_score, balanced_accuracy_score, recall_score, precision_score, f1_score
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.pipeline import make_pipeline
# from sklearn.compose import make_column_transformer
# from sklearn.compose import make_column_selector as selector

Part One - Dataset Basics

For this week's modules I am using the asteroid dataset found on Kaggle here: https://www.kaggle.com/datasets/sakhawat18/asteroid-dataset
(This is a cleaner version of the NASA/JPL datasets used in my portfolio projects, as such thoroughness is not needed for the daily projects)

The target for this exercise will be the Potentially Hazardous Asteroid classification feature. 
Unfortunately (or, fortunately for the Earth) this feature is heavily unbalanced in favor of 'no' so we will need to adjust our metrics.

This is an unbalanced binary classification problem, so we will evaluate balanced accuracy and F1 score, as well as ROC AUC.
We will also evaluate the accuracy of the models trained on synthetically balanced datasets using eg. SMOTE techniques.

The features in this dataset have a lot of redundancy due to having been merged from various sources; additionally there are
many label features which need to be culled for modeling purposes and to prevent cross-leaking. 

Also worthy of note is that the PHA designation is a direct fuction of two features, absolute magnitude ('H') and minimum orbit intercept 
distance ('moid'), so models will eventually be compared between the initial inference with the features included, and ones built without 
those features.

In [8]:
# Wrangling Functions

def wrangle(filepath):

    df = pd.read_csv(filepath, index_col=['pdes']) #pdes = primary designation number
    
    #drop extraneous label/index/constant columns
    labels = ['id', 'spkid', 'full_name', 'name', 'prefix', 'orbit_id', 'equinox', 'class']
    df.drop(columns=labels, inplace=True)
    #drop duplicate date/distance columns
    #NOTE: we want consistent Julian dates & AU distances
    labels= ['epoch_mjd', 'epoch_cal', 'tp_cal', 'per_y', 'moid_ld']
    df.drop(columns=labels, inplace=True)
    #drop leaky columns
    labels = ['neo']
    df.drop(columns=labels, inplace=True)

    #drop rows with no target value
    df.dropna(subset=['pha'], inplace=True)

    #convert pha to tidy numeric binary encoding
    df['pha'] = [1 if flag=='Y' else 0 for flag in df['pha']]

    #calculate missing albedo from magnitude
    abdmask =  (df['albedo'].isna() & ~(df['H'].isna() | df['diameter'].isna()))
    df['albedo'].where(~abdmask, ((1329*(10**((-1)*(df['H']/5)))/df['diameter'])**2), inplace=True) 

    # impute remaining missing albedo with standard value
    # see: https://en.wikipedia.org/wiki/Standard_asteroid_physical_characteristics
    df['albedo'].where(~(df['albedo'].isna()), other=0.1, inplace=True)

    # calculate missing diameter values from magnitude
    diamask = (df['diameter'].isna() & ~(df['H'].isna() | df['albedo'].isna()))
    df['diameter'].where(~diamask, (1329/(np.sqrt(df['albedo'])))*(10**((-1)*(df['H']/5))), inplace=True)

    #impute missing diameter sigma values
    #replacing with maximum error as we are using derived diameter values
    df['diameter_sigma'].where(~(df['diameter_sigma'].isna()), other=df['diameter_sigma'].max(), inplace=True)

    #finally, calculate missing magnitude values from albedo and diameter
    hmask = (df['H'].isna() & ~(df['albedo'].isna() | df['diameter'].isna()))
    df['H'].where(~hmask, 5 * (np.log10((1329/(df['diameter'] * np.sqrt(df['albedo']))))), inplace=True)

    #any remaining missing values must be dropped
    df.dropna(inplace=True)

    return df

In [9]:
df = wrangle(DATA_PATH + "/Asteroids/dataset.csv")

print(df.shape)
print("")
print(df.info())

c:\Users\Jeff\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


(936499, 30)

<class 'pandas.core.frame.DataFrame'>
Index: 936499 entries, 1 to 2678 T-3
Data columns (total 30 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   pha             936499 non-null  int64  
 1   H               936499 non-null  float64
 2   diameter        936499 non-null  float64
 3   albedo          936499 non-null  float64
 4   diameter_sigma  936499 non-null  float64
 5   epoch           936499 non-null  float64
 6   e               936499 non-null  float64
 7   a               936499 non-null  float64
 8   q               936499 non-null  float64
 9   i               936499 non-null  float64
 10  om              936499 non-null  float64
 11  w               936499 non-null  float64
 12  ma              936499 non-null  float64
 13  ad              936499 non-null  float64
 14  n               936499 non-null  float64
 15  tp              936499 non-null  float64
 16  per             936499 non-null  float64
 17 

Part Two - Baseline Modeling

Baseline accuracy of ~0.998 is a bit problematic, so we will try 
1. Different scoring methods
2. Some basic resampling 
before building the initial models

In [10]:
# Create two datasets to model, with/without the H (magnitude) + moid columns
# easy mode = default columns, hard mode = without, model must infer from other data
df_hard = df.drop(columns=['H', 'moid'])

X = df.drop(columns='pha')
y = df['pha']

X_h = df_hard.drop(columns='pha')
y_h = df_hard['pha']

#naive baseline scores
model_dum = DummyClassifier(strategy='prior').fit(X, y)
dum_pred = model_dum.predict(X)
baseline_acc = accuracy_score(y, dum_pred)
print(baseline_acc)
base_bal_acc = balanced_accuracy_score(y, dum_pred)
print(base_bal_acc)


0.9977939111520674
0.5


In [25]:
#split datasets into train, test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_h_train, X_h_test, y_h_train, y_h_test = train_test_split(X_h, y_h, test_size=0.2, random_state=42)

#get majority/minority class splits for resampling
class_mask = (df['pha'] == 1)
X_train_min, X_train_maj = X_train.loc[class_mask], X_train.loc[~class_mask]
X_h_train_min, X_h_train_maj = X_h_train.loc[class_mask], X_h_train.loc[~class_mask]

#evenly over- and under-sample dataset to create balanced training set
pha_y, pha_n = y_train.value_counts(sort=True, ascending=True)

train_midp_sample_1 = X_train_min.sample(((pha_n + pha_y)//2), replace=True)
y_sample_1 = pd.Series([1] * len(train_midp_sample_1))
train_midp_sample_0 = X_train_maj.sample(((pha_n + pha_y)//2)+1, replace=False)
y_sample_0 = pd.Series([0] * len(train_midp_sample_0))
X_train_res =  train_midp_sample_1.append(train_midp_sample_0, ignore_index=True)
y_train_res = pd.concat([y_sample_1, y_sample_0], ignore_index=True)

pha_h_y, pha_h_n = y_h_train.value_counts(sort=True, ascending=True)

train_h_midp_sample_1 = X_h_train_min.sample(((pha_n + pha_y)//2), replace=True)
y_h_sample_1 = pd.Series([1] * len(train_h_midp_sample_1))
train_h_midp_sample_0 = X_h_train_maj.sample(((pha_h_n + pha_h_y)//2)+1, replace=False)
y_h_sample_0 = pd.Series([0] * len(train_h_midp_sample_0))
X_h_train_res =  train_h_midp_sample_1.append(train_h_midp_sample_0, ignore_index=True)
y_h_train_res = pd.concat([y_h_sample_1, y_h_sample_0], ignore_index=True)

#check for deformed arrays
print(X_train.shape)
print(X_train_res.shape)
print(y_train.shape)
print(y_train_res.shape)

print(X_h_train.shape)
print(X_h_train_res.shape)
print(y_h_train.shape)
print(y_h_train_res.shape)


(749199, 29)
(749199, 29)
(749199,)
(749199,)
(749199, 27)
(749199, 27)
(749199,)
(749199,)


In [30]:
# Initial models

model_lr = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))
model_rf = make_pipeline(RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42))

print("######### Original Dataset Distributions ########")

print("*Full Data Columns*")

model_lr.fit(X_train, y_train)
lr_acc = accuracy_score(y_test, model_lr.predict(X_test))
print("LR Model Accuracy: ", lr_acc)
lr_acc_bal = balanced_accuracy_score(y_test, model_lr.predict(X_test))
print("LR Model Balanced Accuracy: ", lr_acc_bal)

model_rf.fit(X_train, y_train)
rf_acc = accuracy_score(y_test, model_rf.predict(X_test))
print("RF Model Accuracy: ", rf_acc)
rf_acc_bal = balanced_accuracy_score(y_test, model_rf.predict(X_test))
print("RF Model Balanced Accuracy: ", rf_acc_bal)

print("*Limited Data Columns*")

model_lr.fit(X_h_train, y_h_train)
lr_h_acc = accuracy_score(y_h_test, model_lr.predict(X_h_test))
print("LR Model Accuracy: ", lr_h_acc)
lr_h_acc_bal = balanced_accuracy_score(y_h_test, model_lr.predict(X_h_test))
print("LR Model Balanced Accuracy: ", lr_h_acc_bal)

model_rf.fit(X_h_train, y_h_train)
rf_h_acc = accuracy_score(y_h_test, model_rf.predict(X_h_test))
print("RF Model Accuracy: ", rf_h_acc)
rf_h_acc_bal = balanced_accuracy_score(y_h_test, model_rf.predict(X_h_test))
print("RF Model Balanced Accuracy: ", rf_h_acc_bal)

print("######### Resampled Dataset Distributions ########")

print("*Full Data Columns*")

model_lr.fit(X_train_res, y_train_res)
lr_acc = accuracy_score(y_test, model_lr.predict(X_test))
print("LR Model Accuracy: ", lr_acc)
lr_acc_bal = balanced_accuracy_score(y_test, model_lr.predict(X_test))
print("LR Model Balanced Accuracy: ", lr_acc_bal)

model_rf.fit(X_train_res, y_train_res)
rf_acc = accuracy_score(y_test, model_rf.predict(X_test))
print("RF Model Accuracy: ", rf_acc)
rf_acc_bal = balanced_accuracy_score(y_test, model_rf.predict(X_test))
print("RF Model Balanced Accuracy: ", rf_acc_bal)

print("*Limited Data Columns*")

model_lr.fit(X_h_train_res, y_h_train_res)
lr_h_acc = accuracy_score(y_h_test, model_lr.predict(X_h_test))
print("LR Model Accuracy: ", lr_h_acc)
lr_h_acc_bal = balanced_accuracy_score(y_h_test, model_lr.predict(X_h_test))
print("LR Model Balanced Accuracy: ", lr_h_acc_bal)

model_rf.fit(X_h_train_res, y_h_train_res)
rf_h_acc = accuracy_score(y_h_test, model_rf.predict(X_h_test))
print("RF Model Accuracy: ", rf_h_acc)
rf_h_acc_bal = balanced_accuracy_score(y_h_test, model_rf.predict(X_h_test))
print("RF Model Balanced Accuracy: ", rf_h_acc_bal)


######### Original Dataset Distributions ########
*Full Data Columns*
LR Model Accuracy:  0.9982594767752269
LR Model Balanced Accuracy:  0.6768255810981356
RF Model Accuracy:  0.999893219434063
RF Model Balanced Accuracy:  0.9828633334716426
*Limited Data Columns*
LR Model Accuracy:  0.9974319273892152
LR Model Balanced Accuracy:  0.524940230108523
RF Model Accuracy:  0.9983342231713828
RF Model Balanced Accuracy:  0.6985016999579061
######### Resampled Dataset Distributions ########
*Full Data Columns*
LR Model Accuracy:  0.9938013881473572
LR Model Balanced Accuracy:  0.9957545526356983
RF Model Accuracy:  0.9998985584623599
RF Model Balanced Accuracy:  0.9965325301229551
*Limited Data Columns*
LR Model Accuracy:  0.9791510945008008
LR Model Balanced Accuracy:  0.9895511125857585
RF Model Accuracy:  0.9978056593699947
RF Model Balanced Accuracy:  0.8736238165453968
